# Competition

# Task Overview
You are given a dataset of top news (of a day) and want to predict the movement (1 for up and 0 for down) of the market value.

Download the data from [competition page](https://www.kaggle.com/t/664260fab9b04f699426b48a29ff7d05). This is also where you will upload your submissions.

You need to improve the accuracy of the model as much as you can.

## Rules:
1. Do not use any external data **NOR** models pre-trained on other datasets
2. Use the test set **ONLY** to get predictions for your model. For example, do not use it to compute statistics or features (e.g. learning preprocessing).
3. Do not use deep learning models for a fair competition
4. Don't cheat :)

## Hints
Here are several techniques that you can use:

1. **Tune your hyper-parameters** Try `GridSerachCV` function from `sklearn.model_selection` to find the best set of hyperparameters.
2. **Feature engineering** Play with the representation of the textual data. We only tried one, but there are more (e.g. TF-IDF Vectorizer is another powerful method to transform text to a vector, taking into account the rareness of the words across the texts). Also do not hesitate to play with the arguments of the *Vectorizers*. 
3. **Change your model** You are not restricted to train `LogisticRegression` only. You can use whatever algorithm you're already familiar with. Moreover, you can use the algorithms that you get to know during these 3 weeks of solving this assignment. E.g. give *RandomForests* a try!
4. **Use date** You can also use the date as extra features, think how you can use it and look for some patterns!
5. **Combine multiple models** You can train multiple models and use their individual predictions to produce a final, improved prediction.

## Scoring rules [16 points + 20 bonus points]
You have until **22.11.2023** to submit your tuned solutions.  
**You also need to submit the code for your best solution before the deadline.**

### **Part of the Assignment grade: [16 points]**
You need to beat two thresholds in order to get a full set of points for the assignment:

- You get **4 points** if you get at least 55% on the public board (`Super Easy Baseline`).

- You get another **12 points** if you beat the **easy baseline** - 58%. (We also added two hard baselines just for a point of reference)

### **Bonus points [up to 20 points]**
- **Top-5** on the final leaderboard get **20 bonus points**

- **Top-10** on the final leaderboard get **15 bonus points**

- **Top-15** on the final leaderboard get **10 bonus points**

- **Top-25** on the final leaderboard get **5 bonus points**

In [353]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train['Date'] = pd.to_datetime(train['Date'], utc=True)
test['Date'] = pd.to_datetime(test['Date'], utc=True)

train['Day'] = train['Date'].dt.day
train['DayOfYear'] = train['Date'].dt.dayofyear

test['Day'] = test['Date'].dt.day
test['DayOfYear'] = test['Date'].dt.dayofyear

In [354]:
train.head(3)  # look at the training data

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Day,DayOfYear
0,2008-08-08 00:00:00+00:00,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",8,221
1,2008-08-11 00:00:00+00:00,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",11,224
2,2008-08-12 00:00:00+00:00,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",12,225


In [355]:
test.head(3)  # and the test

,ID,Date,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Day,DayOfYear
0,0,2015-01-02 00:00:00+00:00,Most cases of cancer are the result of sheer b...,Iran dismissed United States efforts to fight ...,Poll: One in 8 Germans would join anti-Muslim ...,UK royal family's Prince Andrew named in US la...,Some 40 asylum-seekers refused to leave the bu...,Pakistani boat blows self up after India navy ...,Sweden hit by third mosque arson attack in a week,940 cars set alight during French New Year,...,Israeli security center publishes names of 50 ...,The year 2014 was the deadliest year yet in Sy...,A Secret underground complex built by the Nazi...,Restrictions on Web Freedom a Major Global Iss...,Austrian journalist Erich Mchel delivered a pr...,Thousands of Ukraine nationalists march in Kiev,Chinas New Years Resolution: No More Harvestin...,Authorities Pull Plug on Russia's Last Politic...,2,2
1,1,2015-01-05 00:00:00+00:00,Moscow-&gt;Beijing high speed train will reduc...,Two ancient tombs were discovered in Egypt on ...,China complains to Pyongyang after N Korean so...,Scotland Headed Towards Being Fossil Fuel-Free...,Prime Minister Shinzo Abe said Monday he will ...,Sex slave at centre of Prince Andrew scandal f...,Gay relative of Hamas founder faces deportatio...,The number of female drug addicts in Iran has ...,...,Blackfield Capital Founder Goes Missing: The v...,Rocket stage crashes back to Earth in rural Ch...,2 Dead as Aircraft Bombs Greek Tanker in Libya...,Belgian murderer Frank Van Den Bleeken to die ...,Czech President criticizes Ukrainian PM; says ...,3 Vietnamese jets join search for 16 missing F...,France seeks end to Russia sanctions over Ukraine,China scraps rare earths caps,5,5
2,2,2015-01-06 00:00:00+00:00,US oil falls below $50 a barrel,"Toyota gives away 5,680 fuel cell patents to b...",Young Indian couple who had been granted polic...,A senior figure in Islamic States self-declare...,Fukushima rice passes radiation tests for 1st ...,Nearly all Spanish parties guilty of financial...,King Abdullah to abdicate Saudi Throne,Taliban Commander Caught Networking On LinkedIn,...,SpaceX Falcon 9 launch and recovery has been a...,CNN: Americans charged in botched Gambia coup,Islamic State 'Police' Official Beheaded.,Libya bans Palestinians from country to preven...,A judicial inquiry was opened in France on Mon...,Video has captured the moment a cameraman was ...,Syria has complained to the United Nations tha...,"Tests over, India set to make the iris of bigg...",6,6


In [356]:
# naively concatenating all the news
X_train = [' '.join(str(x) for x in train.iloc[row,2:27]) + f' {train.iloc[row, 27]} {train.iloc[row, 28]}'
           for row in range(len(train.index))]
X_test = [' '.join(str(x) for x in test.iloc[row,2:27]) + f' {test.iloc[row, 27]} {test.iloc[row, 28]}'
          for row in range(len(test.index))]

creating a local validation set (since we don't know the test labels and we have limited (per day) submissions to kaggle)

In [357]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, train.Label, test_size=0.2, random_state=42)

One needs to transform the data to the format that can be used with the known classifiers.

We need to represent each text as a classifier-friendly representation, for example: bag of words.

Using *CountVectorizer* from *sklearn.feature_extraction.text* we can transform the *news* to a data matrix *X* of shape [num_days, vocabulary_size], where each row represents a single text and each column indicates the number of occurences of a specific word across the dataset.
Notice that the Vectorizer has a lot of useful arguments. These could potentially influence the performance of the models.

In [358]:
# skip common unimportant words in English
# convert all characters to lowercase before vectorizing
# use unigrams and bigrams ngram_range=(1, 2)
# ignore words with frequency lower than min_df

vectorizer = TfidfVectorizer(analyzer='word', lowercase=True, ngram_range=(2, 2), min_df=2)

X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)

In [359]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

model = RandomForestClassifier(criterion='gini', n_estimators=50, min_samples_split=2, random_state=42)
model.fit(X_train, y_train)

preds = model.predict(vectorizer.transform(X_test))

just to have a sense about our simple classifier, we will evaluate it on validation set.

In [360]:
print((model.predict(X_val) == y_val).mean() * 100.0)

53.86996904024768


In [361]:
# creating a submission file for kaggle
pd.DataFrame({'ID': np.arange(len(preds)), 'Label': preds}).to_csv('submission_1gram.csv', index=False)